In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle

In [ ]:
!cp '/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/Kaggle Notebooks/kaggle.json' ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d kristianfrossos/cicddos2019

100% 3.09G/3.10G [00:37<00:00, 115MB/s]
100% 3.10G/3.10G [00:37<00:00, 88.1MB/s]


In [ ]:
!ls /content

cicddos2019.zip  drive	sample_data


In [ ]:
!unzip -q cicddos2019.zip

In [ ]:
!ls cicddos2019

ls: cannot access 'cicddos2019': No such file or directory


In [ ]:
!pip install keras==3.1.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 12.1 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [6]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.models import load_model
from keras.layers import Layer
from collections import Counter

In [7]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("GPU device not found. Using CPU instead.")

Default GPU Device: /device:GPU:0


In [ ]:
labels = {
    '0': "BENIGN",
    '1': "DrDoS_DNS",
    '2': "DrDoS_LDAP",
    '3': "DrDoS_MSSQL",
    '4': "DrDoS_NTP",
    '5': "DrDoS_NetBIOS",
    '6': "DrDoS_SNMP",
    '7': "DrDoS_SSDP",
    '8': "DrDoS_UDP",
    '9': "LDAP",
    '10': "MSSQL",
    '11': "NetBIOS",
    '12': "Portmap",
    '13': "Syn",
    '14': "UDP",
    '15': "UDP-lag",
    '16': "UDPLag",
    '17': "WebDDoS",
    '18': "vsi"
}

In [ ]:
def load_file(path):
    data = pd.read_csv(path)
    fields=['src_port', 'dst_port',
       'protocol','flow_duration', 'flow_byts_s', 'flow_pkts_s',
       'fwd_pkts_s', 'bwd_pkts_s', 'tot_fwd_pkts', 'tot_bwd_pkts',
       'totlen_fwd_pkts', 'totlen_bwd_pkts', 'fwd_pkt_len_max',
       'fwd_pkt_len_min', 'fwd_pkt_len_mean', 'fwd_pkt_len_std',
       'bwd_pkt_len_max', 'bwd_pkt_len_min', 'bwd_pkt_len_mean',
       'bwd_pkt_len_std', 'pkt_len_max', 'pkt_len_min', 'pkt_len_mean',
       'pkt_len_std', 'pkt_len_var', 'fwd_header_len', 'bwd_header_len', 'flow_iat_mean',
       'flow_iat_max', 'flow_iat_min', 'flow_iat_std', 'fwd_iat_tot',
       'fwd_iat_max', 'fwd_iat_min', 'fwd_iat_mean', 'fwd_iat_std',
       'bwd_iat_tot', 'bwd_iat_max', 'bwd_iat_min', 'bwd_iat_mean',
       'bwd_iat_std', 'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags',
       'bwd_urg_flags', 'fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
       'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt', 'ece_flag_cnt',
       'down_up_ratio', 'pkt_size_avg', 'init_fwd_win_byts',
       'init_bwd_win_byts', 'active_max', 'active_min', 'active_mean',
       'active_std', 'idle_max', 'idle_min', 'idle_mean', 'idle_std',
       'fwd_byts_b_avg', 'fwd_pkts_b_avg', 'bwd_byts_b_avg', 'bwd_pkts_b_avg',
       'fwd_blk_rate_avg', 'bwd_blk_rate_avg', 'fwd_seg_size_avg',
       'bwd_seg_size_avg', 'cwe_flag_count', 'subflow_fwd_pkts',
       'subflow_bwd_pkts', 'subflow_fwd_byts', 'subflow_bwd_byts']
    data=data[fields]
    return data

In [ ]:
def load_file(path):
    data = pd.read_csv(path)
    fields=[' Source Port',' Destination Port',' Protocol',' Flow Duration','Flow Bytes/s',' Flow Packets/s',
        'Fwd Packets/s',' Bwd Packets/s',' Total Fwd Packets',' Total Backward Packets','Total Length of Fwd Packets',' Total Length of Bwd Packets',' Fwd Packet Length Max',
       ' Fwd Packet Length Min',' Fwd Packet Length Mean', ' Fwd Packet Length Std','Bwd Packet Length Max',' Bwd Packet Length Min',' Bwd Packet Length Mean',' Bwd Packet Length Std',
        ' Max Packet Length',' Min Packet Length',' Packet Length Mean',' Packet Length Std',' Packet Length Variance',' Fwd Header Length',' Bwd Header Length',' Flow IAT Mean',
       ' Flow IAT Max',' Flow IAT Min',' Flow IAT Std','Fwd IAT Total',' Fwd IAT Max',' Fwd IAT Min',' Fwd IAT Mean',' Fwd IAT Std','Bwd IAT Total',' Bwd IAT Max',' Bwd IAT Min',
       ' Bwd IAT Mean',' Bwd IAT Std','Fwd PSH Flags',' Bwd PSH Flags',' Fwd URG Flags',' Bwd URG Flags','FIN Flag Count',' SYN Flag Count',' RST Flag Count',' PSH Flag Count',
        ' ACK Flag Count',' URG Flag Count',' ECE Flag Count',' Down/Up Ratio',' Average Packet Size','Init_Win_bytes_forward',' Init_Win_bytes_backward',' Active Max',' Active Min',
        'Active Mean',' Active Std',' Idle Max',' Idle Min','Idle Mean', ' Idle Std','Fwd Avg Bytes/Bulk',' Fwd Avg Packets/Bulk',' Bwd Avg Bytes/Bulk',' Bwd Avg Packets/Bulk',
        ' Fwd Avg Bulk Rate','Bwd Avg Bulk Rate',' Avg Fwd Segment Size',' Avg Bwd Segment Size',' CWE Flag Count','Subflow Fwd Packets',' Subflow Bwd Packets',' Subflow Fwd Bytes',
        ' Subflow Bwd Bytes',' Label']
    data=data[fields]
    return data

In [ ]:
def format_3d(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], X.shape[1], 1))

In [ ]:
def preprocessing(samples):
  samples = samples.replace('Infinity','0')
  samples = samples.replace(np.inf,0)
  #samples = samples.replace('nan','0')
  samples['flow_pkts_s'] = pd.to_numeric(samples['flow_pkts_s'])

  samples['flow_byts_s'] = samples['flow_byts_s'].fillna(0)
  samples['flow_byts_s'] = pd.to_numeric(samples['flow_byts_s'])
  return samples


In [ ]:
def preprocessing(samples):
  samples = samples.replace('Infinity','0')
  samples = samples.replace(np.inf,0)
  #samples = samples.replace('nan','0')
  samples[' Flow Packets/s'] = pd.to_numeric(samples[' Flow Packets/s'])

  samples['Flow Bytes/s'] = samples['Flow Bytes/s'].fillna(0)
  samples['Flow Bytes/s'] = pd.to_numeric(samples['Flow Bytes/s'])
  return samples


In [ ]:
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(name="att_weight", shape=(input_shape[-1], 1), initializer="normal")
        self.b = self.add_weight(name="att_bias", shape=(input_shape[1], 1), initializer="zeros")
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        et = tf.squeeze(tf.tanh(tf.matmul(x, self.W) + self.b), axis=-1)
        at = tf.nn.softmax(et)
        at = tf.expand_dims(at, axis=-1)
        output = x * at
        return tf.reduce_sum(output, axis=1)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])


In [ ]:
num_classes=18
def combine_predictions(predictions):
    num_samples = len(predictions[0])
    combined_predictions = np.zeros((num_samples, num_classes))

    for i in range(num_samples):
        combined_predictions[i] = sum(predictions[j][i] for j in range(len(predictions)))

    final_predictions = np.argmax(combined_predictions, axis=1)

    return final_predictions



In [ ]:
data1=load_file('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/syn.csv')
data1=preprocessing(data1)
data1=format_3d(data1)

In [ ]:
model1 = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel1.keras', custom_objects={'AttentionLayer': AttentionLayer})
model2 = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel2.keras', custom_objects={'AttentionLayer': AttentionLayer})
model3 = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel3.keras', custom_objects={'AttentionLayer': AttentionLayer})
model4 = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel4.keras', custom_objects={'AttentionLayer': AttentionLayer})
model5 = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel5.keras', custom_objects={'AttentionLayer': AttentionLayer})

In [ ]:
model1_predictions = model1.predict(data1)
model2_predictions = model2.predict(data1)
model3_predictions = model3.predict(data1)
model4_predictions = model4.predict(data1)
model5_predictions = model5.predict(data1)

357/357 ━━━━━━━━━━━━━━━━━━━━ 9s 24ms/step
357/357 ━━━━━━━━━━━━━━━━━━━━ 8s 20ms/step
357/357 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step
357/357 ━━━━━━━━━━━━━━━━━━━━ 9s 23ms/step
357/357 ━━━━━━━━━━━━━━━━━━━━ 8s 22ms/step


In [ ]:
# Combine predictions from all models
all_predictions = [model1_predictions, model2_predictions, model3_predictions, model4_predictions, model5_predictions]
final_predictions = combine_predictions(all_predictions)
final_predictions_labels = [labels[str(prediction)] for prediction in final_predictions]
# print("Final predictions (summed and maximum likelihood):")
# print(final_predictions_labels)

In [ ]:
label_counts = Counter(final_predictions_labels)

print("Counts of unique labels:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

Counts of unique labels:
BENIGN: 11420
UDP: 2


In [ ]:
max_label, max_count = label_counts.most_common(1)[0]

print("Label with maximum count:")
print(f"{max_label}: {max_count}")

Label with maximum count:
MSSQL: 8756


# Transfer wala


In [ ]:
data=load_file('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/syn.csv')
data=preprocessing(data)
data=format_3d(data)

In [ ]:
model = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/trained_model/Final Trained/transfer_model_saturday.keras', custom_objects={'AttentionLayer': AttentionLayer})

In [ ]:
model_predictions = model.predict(data)
predictions = np.argmax(model_predictions, axis=1)

357/357 ━━━━━━━━━━━━━━━━━━━━ 11s 26ms/step


In [ ]:

label_counts = Counter(predictions)

# print("Counts of unique labels:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

0: 11415
5: 7


In [ ]:

dataset=['/content/CSV-03-11/03-11/Portmap.csv',
         '/content/CSV-03-11/03-11/UDPLag.csv']

In [ ]:
data = pd.concat([load_file(dfp1) for dfp1 in dataset], ignore_index=True)
nlabel=data[' Label'].nunique()
print(nlabel)

<ipython-input-19-fc5369fa1e4d>:2: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)
<ipython-input-19-fc5369fa1e4d>:2: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


5


In [ ]:
from tensorflow.keras.utils import to_categorical


In [ ]:
fdata=preprocessing(data)
x1=fdata.drop(columns=[' Label']).values
x1=format_3d(x1)
y1=fdata[' Label']
y1 = to_categorical(y1, num_classes=18)
print(y1)

ValueError: invalid literal for int() with base 10: 'BENIGN'

In [ ]:
import numpy as np
from tensorflow.keras.models import load_model

# Load your pre-trained Keras models
model_A = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel1.keras', custom_objects={'AttentionLayer': AttentionLayer})
model_B = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel2.keras', custom_objects={'AttentionLayer': AttentionLayer})
model_C = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel3.keras', custom_objects={'AttentionLayer': AttentionLayer})
model_D = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel4.keras', custom_objects={'AttentionLayer': AttentionLayer})
model_E = load_model('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/Ensemble/EnsembleModel5.keras', custom_objects={'AttentionLayer': AttentionLayer})


In [ ]:

# Assuming you have your test data stored in X_test (3D array with dimensions: samples, time_steps, features)

# Step 1: Make predictions using the pre-trained models
preds_A = model_A.predict(X_test)
preds_B = model_B.predict(X_test)
preds_C = model_C.predict(X_test)
preds_D = model_D.predict(X_test)
preds_E = model_E.predict(X_test)




In [ ]:
# Step 2: Create stacked dataset
stacked_X_test = np.concatenate((preds_A, preds_B, preds_C, preds_D, preds_E), axis=2)

# Assuming you have your training data stored in X_train (3D array with dimensions: samples, time_steps, features)
# Assuming you have your training labels stored in y_train


In [ ]:

# Step 3: Make predictions using the pre-trained models for training data
stacked_preds_train_A = model_A.predict(X_train)
stacked_preds_train_B = model_B.predict(X_train)
stacked_preds_train_C = model_C.predict(X_train)
stacked_preds_train_D = model_D.predict(X_train)
stacked_preds_train_E = model_E.predict(X_train)



In [ ]:
# Step 4: Create stacked dataset for training meta-model
stacked_X_train = np.concatenate((stacked_preds_train_A, stacked_preds_train_B, stacked_preds_train_C, stacked_preds_train_D, stacked_preds_train_E), axis=2)



In [ ]:
# Step 5: Train meta-model on the stacked dataset
from sklearn.linear_model import LogisticRegression

meta_model = LogisticRegression()
meta_model.fit(stacked_X_train.reshape(stacked_X_train.shape[0], -1), y_train)


In [ ]:

# Step 6: Make predictions using meta-model
final_preds = meta_model.predict(stacked_X_test.reshape(stacked_X_test.shape[0], -1))



In [ ]:
# Step 7: Evaluate performance
# Assuming you have your test labels stored in y_test
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, final_preds)
print("Accuracy of stacked model:", accuracy)

In [93]:
fields=['src_ip','src_port','dst_ip','dst_port','protocol','timestamp','flow_duration','tot_fwd_pkts','tot_bwd_pkts',
        'totlen_fwd_pkts', 'totlen_bwd_pkts','fwd_pkt_len_max','fwd_pkt_len_min','fwd_pkt_len_mean', 'fwd_pkt_len_std',
         'bwd_pkt_len_max','bwd_pkt_len_min','bwd_pkt_len_mean','bwd_pkt_len_std','flow_byts_s','flow_pkts_s',
         'flow_iat_mean','flow_iat_std','flow_iat_max', 'flow_iat_min','fwd_iat_tot','fwd_iat_mean', 'fwd_iat_std',
         'fwd_iat_max', 'fwd_iat_min','bwd_iat_tot','bwd_iat_mean','bwd_iat_std','bwd_iat_max','bwd_iat_min',
         'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags','bwd_urg_flags', 'fwd_header_len','bwd_header_len','fwd_pkts_s', 'bwd_pkts_s',
         'pkt_len_min','pkt_len_max','pkt_len_mean','pkt_len_std','pkt_len_var','fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
         'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt','cwe_flag_count','ece_flag_cnt','down_up_ratio','pkt_size_avg','fwd_seg_size_avg',
         'bwd_seg_size_avg','fwdheaderlen','fwd_byts_b_avg','fwd_pkts_b_avg', 'fwd_blk_rate_avg','bwd_byts_b_avg','bwd_pkts_b_avg',
         'bwd_blk_rate_avg','subflow_fwd_pkts','subflow_fwd_byts','subflow_bwd_pkts','subflow_bwd_byts','init_fwd_win_byts',
         'init_bwd_win_byts','fwd_act_data_pkts','fwd_seg_size_min','active_mean','active_std','active_max','active_min','idle_mean',
         'idle_std','idle_max','idle_min','Inbound']



In [94]:
len(fields)

84

# Kaggle Wala Model

In [9]:
import pandas as pd
import numpy as np
from collections import Counter

In [2]:
# mult = 5

cols= ['src_port','dst_port','protocol','timestamp','flow_duration','tot_fwd_pkts','tot_bwd_pkts',
        'totlen_fwd_pkts', 'totlen_bwd_pkts','fwd_pkt_len_max','fwd_pkt_len_min','fwd_pkt_len_mean', 'fwd_pkt_len_std',
         'bwd_pkt_len_max','bwd_pkt_len_min','bwd_pkt_len_mean','bwd_pkt_len_std','flow_byts_s','flow_pkts_s',
         'flow_iat_mean','flow_iat_std','flow_iat_max', 'flow_iat_min','fwd_iat_tot','fwd_iat_mean', 'fwd_iat_std',
         'fwd_iat_max', 'fwd_iat_min','bwd_iat_tot','bwd_iat_mean','bwd_iat_std','bwd_iat_max','bwd_iat_min',
         'fwd_psh_flags', 'bwd_psh_flags', 'fwd_urg_flags','bwd_urg_flags', 'fwd_header_len','bwd_header_len','fwd_pkts_s', 'bwd_pkts_s',
         'pkt_len_min','pkt_len_max','pkt_len_mean','pkt_len_std','pkt_len_var','fin_flag_cnt', 'syn_flag_cnt', 'rst_flag_cnt',
         'psh_flag_cnt', 'ack_flag_cnt', 'urg_flag_cnt','cwe_flag_count','ece_flag_cnt','down_up_ratio','pkt_size_avg','fwd_seg_size_avg',
         'bwd_seg_size_avg','fwd_byts_b_avg','fwd_pkts_b_avg', 'fwd_blk_rate_avg','bwd_byts_b_avg','bwd_pkts_b_avg',
         'bwd_blk_rate_avg','subflow_fwd_pkts','subflow_fwd_byts','subflow_bwd_pkts','subflow_bwd_byts','init_fwd_win_byts',
         'init_bwd_win_byts','fwd_act_data_pkts','fwd_seg_size_min','active_mean','active_std','active_max','active_min','idle_mean',
         'idle_std','idle_max','idle_min']

def load_file(path):
    data = pd.read_csv(path)
    data=data[cols]
    # data['Inbound']=1
    print(data.shape)
    return data

In [82]:
# mult = 5

cols= [' Source Port', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio',
       ' Average Packet Size', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
       ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
       ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean',
       ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
       ' Idle Max', ' Idle Min']

def load_file(path):
    data = pd.read_csv(path)
    data=data[cols]
    # data['Inbound']=1
    print(data.shape)
    return data

In [3]:
def load_model(name):
    from keras.models import model_from_json

    arq_json = '/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/' + name + '.json'
    json_file = open(arq_json,'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    arq_h5 = '/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/03-111/FinalDataset/Notebooks/TrainedModels/' + name + '.h5'
    loaded_model.load_weights(arq_h5)

    print('Model loaded')

    return loaded_model

In [4]:
def string2numeric_hash(text):
    import hashlib
    return int(hashlib.md5(text.encode()).hexdigest()[:8], 16)

def preprocessing(samples):
  # Flows Packet/s e Bytes/s - Replace infinity by 0
  samples = samples.replace('Infinity','0')
  samples = samples.replace(np.inf,0)
  #samples = samples.replace('nan','0')
  samples['flow_pkts_s'] = pd.to_numeric(samples['flow_pkts_s'])
  samples['flow_byts_s'] = samples['flow_byts_s'].fillna(0)
  samples['flow_byts_s'] = pd.to_numeric(samples['flow_byts_s'])
  samples['timestamp'] = samples['timestamp'].apply(lambda x: x.split()[1])  # Extracting only the time part
  samples['timestamp'] = samples['timestamp'].apply(string2numeric_hash)
  return samples

In [88]:
def string2numeric_hash(text):
    import hashlib
    return int(hashlib.md5(text.encode()).hexdigest()[:8], 16)

def preprocessing(samples):
  # Flows Packet/s e Bytes/s - Replace infinity by 0
  samples = samples.replace('Infinity','0')
  samples = samples.replace(np.inf,0)
  #samples = samples.replace('nan','0')
  samples[' Flow Packets/s'] = pd.to_numeric(samples[' Flow Packets/s'])
  samples['Flow Bytes/s'] = samples['Flow Bytes/s'].fillna(0)
  samples['Flow Bytes/s'] = pd.to_numeric(samples['Flow Bytes/s'])
  samples[' Timestamp'] = samples[' Timestamp'].apply(lambda x: x.split()[1])  # Extracting only the time part
  samples[' Timestamp'] = samples[' Timestamp'].apply(string2numeric_hash)
  return samples

In [5]:
def format_3d(df):
    X = np.array(df)
    return np.reshape(X, (X.shape[0], X.shape[1], 1))

In [6]:
model=load_model('lstm_final_adam_50e')

Model loaded


In [86]:
data=load_file('/content/drive/MyDrive/MachineLearningCSV/DeepDefense-master/Datasets/DNS-testing.csv')

(34020, 80)


<ipython-input-82-353e1b967d94>:32: DtypeWarning: Columns (85) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(path)


In [89]:
data=preprocessing(data)

In [90]:
ReshapedData=format_3d(data)

In [91]:
prediction=model.predict(ReshapedData)

1064/1064 [==============================] - 11s 10ms/step


In [92]:
predictions = np.argmax(prediction, axis=1)

In [93]:
label_counts = Counter(predictions)

# print("Counts of unique labels:")
for label, count in label_counts.items():
    print(f"{label}: {count}")

1: 11324
6: 4290
8: 13214
3: 807
0: 3302
14: 216
4: 159
9: 261
2: 40
5: 120
13: 38
16: 14
11: 31
7: 34
10: 8
15: 160
17: 1
12: 1


In [ ]:
len(cols)